In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
df = pd.read_csv("./week2/data.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

In [3]:
df = df[['Make', 'Model', 'Year', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Vehicle Style',
       'highway MPG', 'city mpg', 'MSRP']]
df = df.rename(columns={'MSRP': "price"})

df.head()

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [4]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [5]:
df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
price                 0
dtype: int64

In [6]:
df = df.fillna(0)

In [7]:
df.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
price                0
dtype: int64

### -------------------------------------- Question 1 --------------------------------------

In [8]:
df.transmission_type.mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

### -------------------------------------- Question 2 --------------------------------------

In [9]:
df.corr(numeric_only=True)

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


### -------------------------------------- Question 3 --------------------------------------

In [10]:
mean_price = df.price.mean()
mean_price

40594.737032063116

In [11]:
df["above_average"] = (df.price > mean_price)
df["above_average"] = df["above_average"].astype('int')

del df['price']

In [12]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.replace(' ', '_')

In [13]:
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,above_average
0,BMW,1_Series_M,2011,335.0,6.0,MANUAL,Coupe,26,19,1
1,BMW,1_Series,2011,300.0,6.0,MANUAL,Convertible,28,19,1
2,BMW,1_Series,2011,300.0,6.0,MANUAL,Coupe,28,20,0
3,BMW,1_Series,2011,230.0,6.0,MANUAL,Coupe,28,18,0
4,BMW,1_Series,2011,230.0,6.0,MANUAL,Convertible,28,18,0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score

In [15]:
df_full_train, df_test= train_test_split(df, test_size=0.2, random_state=42)

In [16]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [17]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [18]:
y_train = df_train["above_average"].values
y_val = df_val["above_average"].values
y_test = df_test["above_average"].values

In [19]:
del df_train["above_average"]
del df_val["above_average"]
del df_test["above_average"]

In [20]:
def mutual_score(series):
    return mutual_info_score(series, df_full_train.above_average)
    
mi = df_full_train[categorical_columns].apply(mutual_score)
round(mi, 2)

make                 0.24
model                0.46
transmission_type    0.02
vehicle_style        0.08
dtype: float64

### -------------------------------------- Question 4 --------------------------------------

In [21]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [22]:
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [23]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

In [24]:
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [25]:
model.predict(X_val)


array([0, 1, 0, ..., 0, 1, 1])

In [26]:
model.score(X_val, y_val).round(2)

0.93

### -------------------------------------- Question 5 --------------------------------------

In [27]:
dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
original_accuracy = model.score(X_val, y_val)
original_accuracy

0.9345362987830466

In [28]:
features = list(df_train.columns)
accuracies = {}
for feature in features:
    dv = DictVectorizer(sparse=False)
    
    df_train_2 = df_train.copy() 
    del df_train_2[feature]
    
    train_dict = df_train_2.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    df_val_2 = df_val.copy() 
    del df_val_2[feature]
    
    val_dict = df_val_2.to_dict(orient='records')
    X_val = dv.transform(val_dict)

    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    accuracies[feature] = model.score(X_val, y_val)


In [29]:
accuracies

{'make': 0.9471254720939991,
 'model': 0.9240453210239195,
 'year': 0.9479647503147294,
 'engine_hp': 0.9206882081409987,
 'engine_cylinders': 0.9458665547629039,
 'transmission_type': 0.9429290809903483,
 'vehicle_style': 0.9441879983214435,
 'highway_mpg': 0.9467058329836341,
 'city_mpg': 0.9324381032312211}

In [30]:
accuracy_differences = {}
for key, value in accuracies.items():
    accuracy_differences[key] = (value-original_accuracy).round(2)

In [31]:
accuracy_differences

{'make': 0.01,
 'model': -0.01,
 'year': 0.01,
 'engine_hp': -0.01,
 'engine_cylinders': 0.01,
 'transmission_type': 0.01,
 'vehicle_style': 0.01,
 'highway_mpg': 0.01,
 'city_mpg': -0.0}